In [1]:
import os
import pickle
import pandas as pd
from pickle import Pickler
import git
import subprocess
from os import getcwd
from os import chdir
from subprocess import Popen
from subprocess import PIPE, STDOUT
import shutil
import xml.etree.ElementTree as ET
import json
import tqdm.auto as tqdm

In [2]:
ROOT_PATH = getcwd()

# Bitcoin

In [2]:
repo_name = 'bitcoin/bitcoin'

In [3]:
folder = repo_name.split('/')[-1]
repo = git.Repo.clone_from('https://github.com/'+repo_name+'.git', folder)

In [27]:
clone_snippets_with_vuln = dict()
idx = 0

key_n = 0

print("Key "+str(key_n))
code_context = []
path = 'src/qt/guiutil.h'
print(path)

chdir(ROOT_PATH+'/'+folder)

proc = subprocess.run(["git", 
                       "log",
                       '--pretty=format:%H',
                       "--",
                       path], 
                      capture_output=True) 

commits = proc.stdout.decode('utf-8').split('\n')
commits.reverse()
found = False
for commit in commits:
    repo.git.checkout(commit)
    
    with open(path, 'r') as f:
        content = f.readlines()
        for line in content:
            if 'stackoverflow' in line:
                print(commit)
                print(line)
                line_index = content.index(line)
                print(line_index)
                found = True
                break;
    if found:
        break;

print("Get date for commit "+commit)
with Popen(
    args=['git', 'show', '-s', '--format=%ci' , commit],
    shell=False,
    stdout=PIPE,
    bufsize=1,
    universal_newlines=True,
) as process:

    for line in process.stdout:
        date = line.strip()
        print(date)

new_commit_index = commits.index(commit)
old_commit_index = new_commit_index-1
proc = subprocess.run(["git", 
                       "diff",
                       commits[old_commit_index]+".."+commits[new_commit_index],
                       '--',
                        path
                      ],
                      capture_output=True)

lines_diff = proc.stdout.decode('utf-8').split('\n')
code_context = []
for line in lines_diff:
    if line.startswith('+') and not line.startswith('+++'):
        code_context.append(line[1:])
clone_snippets_with_vuln[idx]={'repo': repo_name, 'origin_commit': commit, 'file': path, 
                               'code_context': ''.join(code_context), 'commit_date': date}

idx = idx + 1
print()
key_n = key_n + 1

Key 0
src/qt/guiutil.h
5659e73493fcdfb5d0cb9d686c24c4fbe1c217ed
     * https://stackoverflow.com/questions/21646467/how-to-execute-a-functor-or-a-lambda-in-a-given-thread-in-qt-gcd-style

346
Get date for commit 5659e73493fcdfb5d0cb9d686c24c4fbe1c217ed
2020-11-25 16:12:23 +0200



In [28]:
clone_snippets_with_vuln

{0: {'repo': 'bitcoin/bitcoin',
  'origin_commit': '5659e73493fcdfb5d0cb9d686c24c4fbe1c217ed',
  'file': 'src/qt/guiutil.h',
  'code_context': "    /**     * Queue a function to run in an object's event loop. This can be     * replaced by a call to the QMetaObject::invokeMethod functor overload after Qt 5.10, but     * for now use a QObject::connect for compatibility with older Qt versions, based on     * https://stackoverflow.com/questions/21646467/how-to-execute-a-functor-or-a-lambda-in-a-given-thread-in-qt-gcd-style     */    template <typename Fn>    void ObjectInvoke(QObject* object, Fn&& function, Qt::ConnectionType connection = Qt::QueuedConnection)    {        QObject source;        QObject::connect(&source, &QObject::destroyed, object, std::forward<Fn>(function), connection);    }",
  'commit_date': '2020-11-25 16:12:23 +0200'}}

In [29]:
clone_snippets_with_vuln_df = pd.DataFrame.from_dict(clone_snippets_with_vuln, orient='index')

In [30]:
clone_snippets_with_vuln_df = clone_snippets_with_vuln_df.drop_duplicates().reset_index(drop=True)
clone_snippets_with_vuln_df

,repo,origin_commit,file,code_context,commit_date
0,bitcoin/bitcoin,5659e73493fcdfb5d0cb9d686c24c4fbe1c217ed,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2020-11-25 16:12:23 +0200


In [85]:
ghCodeSnippetHistory = pd.DataFrame()

In [86]:
repo.git.checkout('master')

"Your branch is up to date with 'origin/master'."

In [87]:
for index in range(clone_snippets_with_vuln_df.shape[0]):
    print('Key_index '+str(index))
    print(clone_snippets_with_vuln_df['file'][index])
    repo.git.checkout('master')
    proc = subprocess.run(["git", 
                           "log",
                           '--pretty=format:%H',
                           "--follow",
                           "--",
                           clone_snippets_with_vuln_df['file'][index]], 
                          capture_output=True) 

    commits = proc.stdout.decode('utf-8').split('\n')
    commits.reverse()

        
    start_from = commits.index(clone_snippets_with_vuln_df['origin_commit'][index])+1

    print("Start from "+str(start_from))
    
    if len(commits)==1:
        print("This is the only commit on this file")
    else:
        try:
            for number in range(start_from, len(commits)):
                repo.git.checkout(commits[number])
                print('Checkout at commit '+commits[number])
                to_delete = []
                add_to_code_context = []
                
                print(commits[number])
                print("Run diff for "+clone_snippets_with_vuln_df['file'][index])
                proc = subprocess.run(["git", 
                                       "diff",
                                       commits[number-1]+".."+commits[number],
                                       '--',
                                        clone_snippets_with_vuln_df['file'][index]
                                      ],
                                      capture_output=True)

                if proc.stdout:
                    deleted = []
                    added = []
                    print("Diff for "+clone_snippets_with_vuln_df['file'][index])
                    with open('diff.txt', 'w', encoding='utf-8') as f:
                        f.write(proc.stdout.decode('utf-8'))
                    with open('diff.txt', 'r') as f:
                        context_index = None
                        context_line = None
                        context_index = None
                        print("Read diff")
                        content = f.readlines()

                        for line in content:
                            if line.startswith('-'):
                                deleted.append(line[1:])
                            if line.startswith('+'):
                                added.append(line[1:])

                        print("Analyze lines added")
                        print(added)
                        for added_line in added:
                            added_line_index = None
                            
                            if not added_line.startswith('++'):
                                print("Check "+added_line)
                
                                with open(clone_snippets_with_vuln_df['file'][index], 'r') as f:
                                    lines_cc = f.readlines();


                                    for x in lines_cc:
                                        x = x.replace('\n', '').replace('\r', '')
                                         
                                        if x.strip() not in ['*/', '', '{', '}', ' */', '/**', '*']  and x in clone_snippets_with_vuln_df['code_context'][index].split('\r\n')[0]:
                                            context_line = x
                                            print(x)
                                            print(clone_snippets_with_vuln_df['code_context'][index])
                                            print('###')
                                            break;
                                    
                                    print(context_line)
                                    print(context_index)
                                    print(added_line)
                                    if context_line:
                                        print("Line of context found")
                                        context_index = lines_cc.index(context_line+'\n')
                                        added_line_index = lines_cc.index(added_line)

                                        print(context_index)
                                        print(added_line_index)
                                        row_diff = context_index - added_line_index
                                        row_diff = abs(row_diff)
                                        print(row_diff)
                                        if row_diff>=0 and row_diff<=3:
                                            print("Add to code context")
                                            add_to_code_context.append(added_line)

                        print("Analyze lines deleted")
                        for line_deleted in deleted:
                            line_deleted = line_deleted.replace('\n', '')
                            if line_deleted:
                                if line_deleted  in clone_snippets_with_vuln_df['code_context'][index]:
                                    if line_deleted and line_deleted != '{' and line_deleted != '}':
                                        print("Append row")

                                        to_delete.append(line_deleted)


                        if not context_line:
                            to_change = ''
                        else:
                            to_change =  clone_snippets_with_vuln_df['code_context'][index]
                            print("Code context updated before")
                            print(to_change)
                            if to_delete:
                                for entry in to_delete:
                                    to_change = to_change.replace(entry, "")

                                print(commits[number])
                                print(to_delete)
                                 #se sono rmaste solo parentesi graffe allora vedi se il file  stato spostato

                                #analyze_moves = False
                                for line in to_change:
                                    if line == '\r' or line == '\n' or line == '{' or line=='}':
                                        to_change = to_change.replace(line, '')
                            if add_to_code_context:
                                string_to_add = ' '.join(add_to_code_context)
                                to_change = string_to_add + to_change

                            print("Code context updated after")
                            print(to_change)

                        with Popen(
                            args=['git', 'show', '-s', '--format=%ci' ,  commits[number]],
                            shell=False,
                            stdout=PIPE,
                            bufsize=1,
                            universal_newlines=True,
                        ) as process:

                            for line in process.stdout:
                                date = line.strip()
                        ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number], 
                                                            'file': clone_snippets_with_vuln_df['file'][index], 
                                                   'code_context': to_change, 
                                                            'commit_date': date}, ignore_index=True)
                        print()

        except UnicodeDecodeError:
            print("Encoding error")

Key_index 0
src/qt/guiutil.h
Start from 108
Checkout at commit 0d5613f9ded846aa20a95c757672e9560c664c4c
0d5613f9ded846aa20a95c757672e9560c664c4c
Run diff for src/qt/guiutil.h
Diff for src/qt/guiutil.h
Read diff
Analyze lines added
['++ b/src/qt/guiutil.h\n', '#include <netaddress.h>\n', '    /** Convert enum Network to QString */\n', '    QString NetworkToQString(Network net);\n', '\n']
Check #include <netaddress.h>

     * Queue a function to run in an object's event loop. This can be
    /**     * Queue a function to run in an object's event loop. This can be     * replaced by a call to the QMetaObject::invokeMethod functor overload after Qt 5.10, but     * for now use a QObject::connect for compatibility with older Qt versions, based on     * https://stackoverflow.com/questions/21646467/how-to-execute-a-functor-or-a-lambda-in-a-given-thread-in-qt-gcd-style     */    template <typename Fn>    void ObjectInvoke(QObject* object, Fn&& function, Qt::ConnectionType connection = Qt::Queued

C:\Users\admin\AppData\Local\Temp\ipykernel_170520\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],



Checkout at commit e0e55060bf697fe8b9ed43ff62383a0451e9c0ce
e0e55060bf697fe8b9ed43ff62383a0451e9c0ce
Run diff for src/qt/guiutil.h
Diff for src/qt/guiutil.h
Read diff
Analyze lines added
['++ b/src/qt/guiutil.h\n', '    /** Convert QString to OS specific boost path through UTF-8 */\n', '    /** Convert OS specific boost path to QString through UTF-8 */\n', '    /** Convert seconds into a QString with days, hours, mins, secs */\n', '    /** Format CNodeStats.nServices bitmask into a user-readable string */\n', '    /** Format a CNodeStats.m_ping_usec into a user-readable string or display N/A, if 0 */\n', '    /** Format a CNodeCombinedStats.nTimeOffset into a user-readable string */\n']
Check     /** Convert QString to OS specific boost path through UTF-8 */

     * Queue a function to run in an object's event loop. This can be
    /**     * Queue a function to run in an object's event loop. This can be     * replaced by a call to the QMetaObject::invokeMethod functor overload after Q

C:\Users\admin\AppData\Local\Temp\ipykernel_170520\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],


Checkout at commit 7e2beab2d28d8ab039d9554744a11592a7d7dc76
7e2beab2d28d8ab039d9554744a11592a7d7dc76
Run diff for src/qt/guiutil.h
Diff for src/qt/guiutil.h
Read diff
Analyze lines added
['++ b/src/qt/guiutil.h\n', '#include <net.h>\n', '#include <QLabel>\n', '    /** Convert enum ConnectionType to QString */\n', '    QString ConnectionTypeToQString(ConnectionType conn_type);\n', '\n']
Check #include <net.h>

     * Queue a function to run in an object's event loop. This can be
    /**     * Queue a function to run in an object's event loop. This can be     * replaced by a call to the QMetaObject::invokeMethod functor overload after Qt 5.10, but     * for now use a QObject::connect for compatibility with older Qt versions, based on     * https://stackoverflow.com/questions/21646467/how-to-execute-a-functor-or-a-lambda-in-a-given-thread-in-qt-gcd-style     */    template <typename Fn>    void ObjectInvoke(QObject* object, Fn&& function, Qt::ConnectionType connection = Qt::QueuedConnecti

...

C:\Users\admin\AppData\Local\Temp\ipykernel_170520\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],


Checkout at commit c378535e28e0c54353fde9c4a66cb3134a3ee742
c378535e28e0c54353fde9c4a66cb3134a3ee742
Run diff for src/qt/guiutil.h
Diff for src/qt/guiutil.h
Read diff
Analyze lines added
['++ b/src/qt/guiutil.h\n', '    /**\n', '     * Extract first suffix from filter pattern "Description (*.foo)" or "Description (*.foo *.bar ...).\n', '     *\n', '     * @param[in] filter Filter specification such as "Comma Separated Files (*.csv)"\n', '     * @return QString\n', '     */\n', '    QString ExtractFirstSuffixFromFilter(const QString& filter);\n', '\n']
Check     /**

None
None
    /**

Check      * Extract first suffix from filter pattern "Description (*.foo)" or "Description (*.foo *.bar ...).

None
None
     * Extract first suffix from filter pattern "Description (*.foo)" or "Description (*.foo *.bar ...).

Check      *

None
None
     *

Check      * @param[in] filter Filter specification such as "Comma Separated Files (*.csv)"

None
None
     * @param[in] filter Filter specification

C:\Users\admin\AppData\Local\Temp\ipykernel_170520\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],


Checkout at commit 306ccd4927a2efe325c8d84be1bdb79edeb29b04
306ccd4927a2efe325c8d84be1bdb79edeb29b04
Run diff for src/qt/guiutil.h
Diff for src/qt/guiutil.h
Read diff
Analyze lines added
['++ b/src/qt/guiutil.h\n', '// Copyright (c) 2011-2022 The Bitcoin Core developers\n']
Check // Copyright (c) 2011-2022 The Bitcoin Core developers

None
None
// Copyright (c) 2011-2022 The Bitcoin Core developers

Analyze lines deleted



C:\Users\admin\AppData\Local\Temp\ipykernel_170520\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],


In [88]:
ghCodeSnippetHistory

,repo,origin_commit,file,code_context,commit_date
0,bitcoin/bitcoin,0d5613f9ded846aa20a95c757672e9560c664c4c,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2020-12-27 14:29:24 +0100
1,bitcoin/bitcoin,e0e55060bf697fe8b9ed43ff62383a0451e9c0ce,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2020-12-27 14:29:26 +0100
2,bitcoin/bitcoin,7e2beab2d28d8ab039d9554744a11592a7d7dc76,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-01-08 15:24:31 +0100
3,bitcoin/bitcoin,4f0961573318271d4d0737b99057f54c188540f7,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-01-10 21:33:10 +0100
4,bitcoin/bitcoin,506e6585a54818d0613a067f91c0bac2f308a48c,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-01-30 20:28:37 +0100
5,bitcoin/bitcoin,3913d1e8c1f604bdd622d5e81e5077ef52b30466,src/qt/guiutil.h,Queue a function to run in an object's event ...,2021-01-31 18:09:30 +0200
6,bitcoin/bitcoin,6fc72bd6f030c8b49e8b9f68188413766a239d73,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-02-06 22:21:16 +0100
7,bitcoin/bitcoin,623de12d040af89ff7c25de6eb0a19c67179dc5f,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-02-21 21:01:03 +0200
8,bitcoin/bitcoin,4d98b401fbd821700f7a792b0a4cb52c9b71bc9f,src/qt/guiutil.h,Queue a function to run in an object's event ...,2021-03-03 09:48:07 -0800
9,bitcoin/bitcoin,af7e365b1516d660d271475fdfe0c20ae09e66a8,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-04-01 03:05:00 +0300


In [92]:
ghCodeSnippetHistory_aftercppcheck = dict()
idx = 0
num = 0


for _, row in ghCodeSnippetHistory.iterrows():
    
    print(_)
    print(row['origin_commit'])
    print(row['file'])
    repo.git.checkout(row['origin_commit'], force=True)
    proc = subprocess.run(["cppcheck", "--language=c++", "--enable=style", "--template={cwe};{file};{line};{severity};{code}", row['file']], capture_output=True)
    result = proc.stderr.decode('utf-8')
    print(proc.stdout)
    print(proc.stderr)

    if proc.stdout.decode('utf-8') == 'cppcheck: error: could not find or open any of the paths given.\r\n':
        break;

    if result:
        items = result.split('\r\n')
        items = list(filter(None, items))
        for x, y in zip(*[iter(items)] * 2):
            row = x + y
            row = row.split(';')

            if not row[2]=='0':
                print("Flaw found at line "+row[2])
                ghCodeSnippetHistory_aftercppcheck[idx] = {
                    'repo': ghCodeSnippetHistory['repo'][_],
                    'origin_commit': ghCodeSnippetHistory['origin_commit'][_],
                    'file': ghCodeSnippetHistory['file'][_],
                    'code_context': ghCodeSnippetHistory['code_context'][_],
                    'commit_date': ghCodeSnippetHistory['commit_date'][_],
                    'line_vuln': row[2],
                    'code_vuln': row[4]
                }
                idx = idx + 1
            else:
                print("Syntax error")
                break;
    else:
        ghCodeSnippetHistory_aftercppcheck[idx] = {
            'repo': ghCodeSnippetHistory['repo'][_],
            'origin_commit': ghCodeSnippetHistory['origin_commit'][_] ,
            'file': ghCodeSnippetHistory['file'][_] ,
            'code_context': ghCodeSnippetHistory['code_context'][_], 
            'commit_date': ghCodeSnippetHistory['commit_date'][_],
            'line_vuln': '',
            'code_vuln': ''
        }
        idx = idx + 1
    num = num + 1
    print() 

0
0d5613f9ded846aa20a95c757672e9560c664c4c
src/qt/guiutil.h
b'Checking src\\qt\\guiutil.h ...\r\n'
b'398;src\\qt\\guiutil.h;281;style;        ItemDelegate(QObject* parent) : QItemDelegate(parent) {}\r\n        ^\r\n'
Flaw found at line 281

1
e0e55060bf697fe8b9ed43ff62383a0451e9c0ce
src/qt/guiutil.h
b'Checking src\\qt\\guiutil.h ...\r\n'
b'398;src\\qt\\guiutil.h;281;style;        ItemDelegate(QObject* parent) : QItemDelegate(parent) {}\r\n        ^\r\n'
Flaw found at line 281

2
7e2beab2d28d8ab039d9554744a11592a7d7dc76
src/qt/guiutil.h
b'Checking src\\qt\\guiutil.h ...\r\n'
b'398;src\\qt\\guiutil.h;285;style;        ItemDelegate(QObject* parent) : QItemDelegate(parent) {}\r\n        ^\r\n'
Flaw found at line 285

3
4f0961573318271d4d0737b99057f54c188540f7
src/qt/guiutil.h
b'Checking src\\qt\\guiutil.h ...\r\n'
b'398;src\\qt\\guiutil.h;285;style;        ItemDelegate(QObject* parent) : QItemDelegate(parent) {}\r\n        ^\r\n'
Flaw found at line 285

4
506e6585a54818d0613a067f91c0bac2f3

In [94]:
ghCodeSnippetHistory_aftercppcheck

{0: {'repo': 'bitcoin/bitcoin',
  'origin_commit': '0d5613f9ded846aa20a95c757672e9560c664c4c',
  'file': 'src/qt/guiutil.h',
  'code_context': "    /**     * Queue a function to run in an object's event loop. This can be     * replaced by a call to the QMetaObject::invokeMethod functor overload after Qt 5.10, but     * for now use a QObject::connect for compatibility with older Qt versions, based on     * https://stackoverflow.com/questions/21646467/how-to-execute-a-functor-or-a-lambda-in-a-given-thread-in-qt-gcd-style     */    template <typename Fn>    void ObjectInvoke(QObject* object, Fn&& function, Qt::ConnectionType connection = Qt::QueuedConnection)    {        QObject source;        QObject::connect(&source, &QObject::destroyed, object, std::forward<Fn>(function), connection);    }",
  'commit_date': '2020-12-27 14:29:24 +0100',
  'line_vuln': '281',
  'code_vuln': '        ItemDelegate(QObject* parent) : QItemDelegate(parent) {}        ^'},
 1: {'repo': 'bitcoin/bitcoin',
  'o

In [95]:
ghCodeSnippetHistory_aftercppcheck = pd.DataFrame.from_dict(ghCodeSnippetHistory_aftercppcheck, orient='index')

In [96]:
ghCodeSnippetHistory_aftercppcheck = ghCodeSnippetHistory_aftercppcheck.reset_index(drop=True)

In [97]:
len(ghCodeSnippetHistory_aftercppcheck)

26

In [98]:
ghCodeSnippetHistory_aftercppcheck

,repo,origin_commit,file,code_context,commit_date,line_vuln,code_vuln
0,bitcoin/bitcoin,0d5613f9ded846aa20a95c757672e9560c664c4c,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2020-12-27 14:29:24 +0100,281,ItemDelegate(QObject* parent) : QItemD...
1,bitcoin/bitcoin,e0e55060bf697fe8b9ed43ff62383a0451e9c0ce,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2020-12-27 14:29:26 +0100,281,ItemDelegate(QObject* parent) : QItemD...
2,bitcoin/bitcoin,7e2beab2d28d8ab039d9554744a11592a7d7dc76,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-01-08 15:24:31 +0100,285,ItemDelegate(QObject* parent) : QItemD...
3,bitcoin/bitcoin,4f0961573318271d4d0737b99057f54c188540f7,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-01-10 21:33:10 +0100,285,ItemDelegate(QObject* parent) : QItemD...
4,bitcoin/bitcoin,506e6585a54818d0613a067f91c0bac2f308a48c,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-01-30 20:28:37 +0100,288,ItemDelegate(QObject* parent) : QItemD...
5,bitcoin/bitcoin,3913d1e8c1f604bdd622d5e81e5077ef52b30466,src/qt/guiutil.h,Queue a function to run in an object's event ...,2021-01-31 18:09:30 +0200,249,ItemDelegate(QObject* parent) : QItemD...
6,bitcoin/bitcoin,6fc72bd6f030c8b49e8b9f68188413766a239d73,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-02-06 22:21:16 +0100,288,ItemDelegate(QObject* parent) : QItemD...
7,bitcoin/bitcoin,623de12d040af89ff7c25de6eb0a19c67179dc5f,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-02-21 21:01:03 +0200,288,ItemDelegate(QObject* parent) : QItemD...
8,bitcoin/bitcoin,4d98b401fbd821700f7a792b0a4cb52c9b71bc9f,src/qt/guiutil.h,Queue a function to run in an object's event ...,2021-03-03 09:48:07 -0800,251,ItemDelegate(QObject* parent) : QItemD...
9,bitcoin/bitcoin,af7e365b1516d660d271475fdfe0c20ae09e66a8,src/qt/guiutil.h,/** * Queue a function to run in an ob...,2021-04-01 03:05:00 +0300,251,ItemDelegate(QObject* parent) : QItemD...


In [100]:
chdir(ROOT_PATH)
file_name = repo_name.replace('/','_')
ghCodeSnippetHistory_aftercppcheck.to_csv('GHCodeSnippetHistory/history_'+file_name+".csv")

# Tensorflow

In [3]:
repo_name = 'tensorflow/tensorflow'

In [4]:
folder = repo_name.split('/')[-1]
repo = git.Repo.clone_from('https://github.com/'+repo_name+'.git', folder)

In [5]:
clone_snippets_with_vuln = dict()
idx = 0

key_n = 0

print("Key "+str(key_n))
code_context = []
path = 'tensorflow/core/framework/variant_op_registry.h'
print(path)

chdir(ROOT_PATH+'/'+folder)

proc = subprocess.run(["git", 
                       "log",
                       '--pretty=format:%H',
                       "--",
                       path], 
                      capture_output=True) 

commits = proc.stdout.decode('utf-8').split('\n')
commits.reverse()
found = False
for commit in commits:
    repo.git.checkout(commit)
    
    with open(path, 'r') as f:
        content = f.readlines()
        for line in content:
            if 'stackoverflow' in line:
                print(commit)
                print(line)
                line_index = content.index(line)
                print(line_index)
                found = True
                break;
    if found:
        break;

print("Get date for commit "+commit)
with Popen(
    args=['git', 'show', '-s', '--format=%ci' , commit],
    shell=False,
    stdout=PIPE,
    bufsize=1,
    universal_newlines=True,
) as process:

    for line in process.stdout:
        date = line.strip()
        print(date)

new_commit_index = commits.index(commit)
old_commit_index = new_commit_index-1
proc = subprocess.run(["git", 
                       "diff",
                       commits[old_commit_index]+".."+commits[new_commit_index],
                       '--',
                        path
                      ],
                      capture_output=True)

lines_diff = proc.stdout.decode('utf-8').split('\n')
code_context = []
for line in lines_diff:
    if line.startswith('+') and not line.startswith('+++'):
        code_context.append(line[1:])
clone_snippets_with_vuln[idx]={'repo': repo_name, 'origin_commit': commit, 'file': path, 
                               'code_context': ''.join(code_context), 'commit_date': date}

idx = idx + 1
print()
key_n = key_n + 1

Key 0
tensorflow/core/framework/variant_op_registry.h
9d84147b0a895ef970f60dde6b78a5bfcc8ba4d7
  // see https://stackoverflow.com/questions/44475317/variadic-template-issue

170
Get date for commit 9d84147b0a895ef970f60dde6b78a5bfcc8ba4d7
2018-01-23 15:13:14 -0800



In [6]:
clone_snippets_with_vuln

{0: {'repo': 'tensorflow/tensorflow',
  'origin_commit': '9d84147b0a895ef970f60dde6b78a5bfcc8ba4d7',
  'file': 'tensorflow/core/framework/variant_op_registry.h',
  'code_context': '  // this breaks by falling victim to "too perfect forwarding"  // see https://stackoverflow.com/questions/44475317/variadic-template-issue  // and references therein  template <typename Op>  struct FuncTuple {    FuncTuple(const Op& op, const StringPiece& dev, const StringPiece& tname)        : op_type_(op), device_(dev), typename_(tname){};    Op op_type_;    StringPiece device_, typename_;  };  //friend declaration for operator==  // needed for clang  template <typename Op>    friend bool operator==(const FuncTuple<Op> &l, const FuncTuple<Op> &r);    template <typename Op>    std::size_t operator()(const FuncTuple<Op>& x) const {      // The hash of an enum is just its value as a std::size_t.      std::size_t ret = static_cast<std::size_t>(x.op_type_);      ret = Hash64Combine(ret, sp_hasher_(x.device_));

In [7]:
clone_snippets_with_vuln_df = pd.DataFrame.from_dict(clone_snippets_with_vuln, orient='index')

In [8]:
clone_snippets_with_vuln_df = clone_snippets_with_vuln_df.drop_duplicates().reset_index(drop=True)
clone_snippets_with_vuln_df

,repo,origin_commit,file,code_context,commit_date
0,tensorflow/tensorflow,9d84147b0a895ef970f60dde6b78a5bfcc8ba4d7,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-01-23 15:13:14 -0800


In [9]:
ghCodeSnippetHistory = pd.DataFrame()

In [10]:
repo.git.checkout('master')

"Your branch is up to date with 'origin/master'."

In [11]:
for index in range(clone_snippets_with_vuln_df.shape[0]):
    print('Key_index '+str(index))
    print(clone_snippets_with_vuln_df['file'][index])
    repo.git.checkout('master')
    proc = subprocess.run(["git", 
                           "log",
                           '--pretty=format:%H',
                           "--follow",
                           "--",
                           clone_snippets_with_vuln_df['file'][index]], 
                          capture_output=True) 

    commits = proc.stdout.decode('utf-8').split('\n')
    commits.reverse()

        
    start_from = commits.index(clone_snippets_with_vuln_df['origin_commit'][index])+1

    print("Start from "+str(start_from))
    
    if len(commits)==1:
        print("This is the only commit on this file")
    else:
        try:
            for number in range(start_from, len(commits)):
                repo.git.checkout(commits[number])
                print('Checkout at commit '+commits[number])
                to_delete = []
                add_to_code_context = []
                
                print(commits[number])
                print("Run diff for "+clone_snippets_with_vuln_df['file'][index])
                proc = subprocess.run(["git", 
                                       "diff",
                                       commits[number-1]+".."+commits[number],
                                       '--',
                                        clone_snippets_with_vuln_df['file'][index]
                                      ],
                                      capture_output=True)

                if proc.stdout:
                    deleted = []
                    added = []
                    print("Diff for "+clone_snippets_with_vuln_df['file'][index])
                    with open('diff.txt', 'w', encoding='utf-8') as f:
                        f.write(proc.stdout.decode('utf-8'))
                    with open('diff.txt', 'r') as f:
                        context_index = None
                        context_line = None
                        context_index = None
                        print("Read diff")
                        content = f.readlines()

                        for line in content:
                            if line.startswith('-'):
                                deleted.append(line[1:])
                            if line.startswith('+'):
                                added.append(line[1:])

                        print("Analyze lines added")
                        print(added)
                        for added_line in added:
                            added_line_index = None
                            
                            if not added_line.startswith('++'):
                                print("Check "+added_line)
                
                                with open(clone_snippets_with_vuln_df['file'][index], 'r') as f:
                                    lines_cc = f.readlines();


                                    for x in lines_cc:
                                        x = x.replace('\n', '').replace('\r', '')
                                         
                                        if x.strip() not in ['*/', '', '{', '}', ' */', '/**', '*']  and x in clone_snippets_with_vuln_df['code_context'][index].split('\r\n')[0]:
                                            context_line = x
                                            print(x)
                                            print(clone_snippets_with_vuln_df['code_context'][index])
                                            print('###')
                                            break;
                                    
                                    print(context_line)
                                    print(context_index)
                                    print(added_line)
                                    if context_line:
                                        print("Line of context found")
                                        context_index = lines_cc.index(context_line+'\n')
                                        added_line_index = lines_cc.index(added_line)

                                        print(context_index)
                                        print(added_line_index)
                                        row_diff = context_index - added_line_index
                                        row_diff = abs(row_diff)
                                        print(row_diff)
                                        if row_diff>=0 and row_diff<=3:
                                            print("Add to code context")
                                            add_to_code_context.append(added_line)

                        print("Analyze lines deleted")
                        for line_deleted in deleted:
                            line_deleted = line_deleted.replace('\n', '')
                            if line_deleted:
                                if line_deleted  in clone_snippets_with_vuln_df['code_context'][index]:
                                    if line_deleted and line_deleted != '{' and line_deleted != '}':
                                        print("Append row")

                                        to_delete.append(line_deleted)


                        if not context_line:
                            to_change = ''
                        else:
                            to_change =  clone_snippets_with_vuln_df['code_context'][index]
                            print("Code context updated before")
                            print(to_change)
                            if to_delete:
                                for entry in to_delete:
                                    to_change = to_change.replace(entry, "")

                                print(commits[number])
                                print(to_delete)
                                 #se sono rmaste solo parentesi graffe allora vedi se il file  stato spostato

                                #analyze_moves = False
                                for line in to_change:
                                    if line == '\r' or line == '\n' or line == '{' or line=='}':
                                        to_change = to_change.replace(line, '')
                            if add_to_code_context:
                                string_to_add = ' '.join(add_to_code_context)
                                to_change = string_to_add + to_change

                            print("Code context updated after")
                            print(to_change)

                        with Popen(
                            args=['git', 'show', '-s', '--format=%ci' ,  commits[number]],
                            shell=False,
                            stdout=PIPE,
                            bufsize=1,
                            universal_newlines=True,
                        ) as process:

                            for line in process.stdout:
                                date = line.strip()
                        ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number], 
                                                            'file': clone_snippets_with_vuln_df['file'][index], 
                                                   'code_context': to_change, 
                                                            'commit_date': date}, ignore_index=True)
                        print()

        except UnicodeDecodeError:
            print("Encoding error")

Key_index 0
tensorflow/core/framework/variant_op_registry.h
Start from 10
Checkout at commit d90054e7c0f41f4bab81df0548577a73b939a87a
d90054e7c0f41f4bab81df0548577a73b939a87a
Run diff for tensorflow/core/framework/variant_op_registry.h
Diff for tensorflow/core/framework/variant_op_registry.h
Read diff
Analyze lines added
['++ b/tensorflow/core/framework/variant_op_registry.h\n', '  // friend declaration for operator==\n', '  friend bool operator==(const FuncTuple<Op>& l, const FuncTuple<Op>& r);\n', '  // container; return the StringPiece pointing to the permanent string\n', '  // location.\n', '        op, " Variant type_name: \'", a.TypeName(),\n', '        "\' for device type: ", device);\n']
Check   // friend declaration for operator==

};
  // this breaks by falling victim to "too perfect forwarding"  // see https://stackoverflow.com/questions/44475317/variadic-template-issue  // and references therein  template <typename Op>  struct FuncTuple {    FuncTuple(const Op& op, const St

C:\Users\admin\AppData\Local\Temp\ipykernel_149780\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],



Checkout at commit 7144571f2fc59c8705e4e3d7b922fa0ebf44f3fa
7144571f2fc59c8705e4e3d7b922fa0ebf44f3fa
Run diff for tensorflow/core/framework/variant_op_registry.h
Diff for tensorflow/core/framework/variant_op_registry.h
Read diff
Analyze lines added
['++ b/tensorflow/core/framework/variant_op_registry.h\n', '        op, " Variant type_name: \'", a.TypeName(), "\' for device type: ",\n', '        device);\n']
Check         op, " Variant type_name: '", a.TypeName(), "' for device type: ",

};
  // this breaks by falling victim to "too perfect forwarding"  // see https://stackoverflow.com/questions/44475317/variadic-template-issue  // and references therein  template <typename Op>  struct FuncTuple {    FuncTuple(const Op& op, const StringPiece& dev, const StringPiece& tname)        : op_type_(op), device_(dev), typename_(tname){};    Op op_type_;    StringPiece device_, typename_;  };  //friend declaration for operator==  // needed for clang  template <typename Op>    friend bool operato

C:\Users\admin\AppData\Local\Temp\ipykernel_149780\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],


Checkout at commit 7817237d5852c7778d3ba03c40f139e6f2c37a76
7817237d5852c7778d3ba03c40f139e6f2c37a76
Run diff for tensorflow/core/framework/variant_op_registry.h
Diff for tensorflow/core/framework/variant_op_registry.h
Read diff
Analyze lines added
['++ b/tensorflow/core/framework/variant_op_registry.h\n', '#ifndef TENSORFLOW_CORE_FRAMEWORK_VARIANT_OP_REGISTRY_H_\n', '#define TENSORFLOW_CORE_FRAMEWORK_VARIANT_OP_REGISTRY_H_\n', '#endif  // TENSORFLOW_CORE_FRAMEWORK_VARIANT_OP_REGISTRY_H_\n']
Check #ifndef TENSORFLOW_CORE_FRAMEWORK_VARIANT_OP_REGISTRY_H_

};
  // this breaks by falling victim to "too perfect forwarding"  // see https://stackoverflow.com/questions/44475317/variadic-template-issue  // and references therein  template <typename Op>  struct FuncTuple {    FuncTuple(const Op& op, const StringPiece& dev, const StringPiece& tname)        : op_type_(op), device_(dev), typename_(tname){};    Op op_type_;    StringPiece device_, typename_;  };  //friend declaration for operator==

...

C:\Users\admin\AppData\Local\Temp\ipykernel_149780\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],



Checkout at commit 603a21810f9e129ce86c3c7cc4a6f37586f791b0
603a21810f9e129ce86c3c7cc4a6f37586f791b0
Run diff for tensorflow/core/framework/variant_op_registry.h
Diff for tensorflow/core/framework/variant_op_registry.h
Read diff
Analyze lines added
['++ b/tensorflow/core/framework/variant_op_registry.h\n', 'const char* VariantUnaryOpToString(VariantUnaryOp op);\n', '\n', 'const char* VariantBinaryOpToString(VariantBinaryOp op);\n', '\n', '    return errors::Internal("No unary variant unary_op function found for op ",\n', '                            VariantUnaryOpToString(op),\n', '                            " Variant type_name: ", v.TypeName(),\n', '                            " for device type: ", device);\n', '    return errors::Internal("No unary variant binary_op function found for op ",\n', '                            VariantBinaryOpToString(op),\n', '                            " Variant type_name: \'", a.TypeName(),\n', '                            "\' for device type: ", de

C:\Users\admin\AppData\Local\Temp\ipykernel_149780\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],



Checkout at commit 39812646c944af98ee9ad1c1ce86eb9d57a1aded
39812646c944af98ee9ad1c1ce86eb9d57a1aded
Run diff for tensorflow/core/framework/variant_op_registry.h
Diff for tensorflow/core/framework/variant_op_registry.h
Read diff
Analyze lines added
['++ b/tensorflow/core/framework/variant_op_registry.h\n', 'class UnaryVariantOpRegistry;\n', 'extern UnaryVariantOpRegistry* UnaryVariantOpRegistryGlobal();\n', '\n', '    return UnaryVariantOpRegistryGlobal();\n']
Check class UnaryVariantOpRegistry;

};
  // this breaks by falling victim to "too perfect forwarding"  // see https://stackoverflow.com/questions/44475317/variadic-template-issue  // and references therein  template <typename Op>  struct FuncTuple {    FuncTuple(const Op& op, const StringPiece& dev, const StringPiece& tname)        : op_type_(op), device_(dev), typename_(tname){};    Op op_type_;    StringPiece device_, typename_;  };  //friend declaration for operator==  // needed for clang  template <typename Op>    friend bo

C:\Users\admin\AppData\Local\Temp\ipykernel_149780\737327823.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number],


In [12]:
ghCodeSnippetHistory

,repo,origin_commit,file,code_context,commit_date
0,tensorflow/tensorflow,d90054e7c0f41f4bab81df0548577a73b939a87a,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-02-07 14:39:49 -0800
1,tensorflow/tensorflow,7144571f2fc59c8705e4e3d7b922fa0ebf44f3fa,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-03-12 19:37:39 -0700
2,tensorflow/tensorflow,7817237d5852c7778d3ba03c40f139e6f2c37a76,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-08-21 16:22:05 -0700
3,tensorflow/tensorflow,36e1a5ea5ba2dd5eaa7f4cfc84a61f8ce3ea20e1,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-09-11 10:51:01 -0700
4,tensorflow/tensorflow,6fa6bd045c98bdc89424a3425e15b5161586a9a7,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-10-15 11:01:32 -0700
5,tensorflow/tensorflow,3f23f4ddeabbdc0704444d84c158bd6c348a9f10,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-10-15 11:22:32 -0700
6,tensorflow/tensorflow,52c8bdba0081e4ed428add13e3c0da7ccbfc8f06,tensorflow/core/framework/variant_op_registry.h,,2019-01-26 10:19:26 -0800
7,tensorflow/tensorflow,063d42680b33321ee8f334b86c7d37cb19ec0281,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2019-06-20 10:29:14 -0700
8,tensorflow/tensorflow,de37b1eaca05431822223e5c996bc08245cf523b,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2020-01-16 13:11:38 -0800
9,tensorflow/tensorflow,775a828aade1b87811875ea0ddfa0554c442654b,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2020-02-27 15:42:16 +0900


In [15]:
ghCodeSnippetHistory_aftercppcheck = dict()
idx = 0
num = 0


for _, row in ghCodeSnippetHistory.iterrows():
    
    print(_)
    print(row['origin_commit'])
    print(row['file'])
    repo.git.checkout(row['origin_commit'], force=True)
    proc = subprocess.run(["cppcheck", "--language=c++", "--enable=style", "--template={cwe};{file};{line};{severity};{code}", row['file']], capture_output=True)
    result = proc.stderr.decode('utf-8')
    print(proc.stdout)
    print(proc.stderr)

    if proc.stdout.decode('utf-8') == 'cppcheck: error: could not find or open any of the paths given.\r\n':
        break;

    if result:
        items = result.split('\r\n')
        items = list(filter(None, items))
        for x, y in zip(*[iter(items)] * 2):
            row = x + y
            row = row.split(';')

            if not row[2]=='0':
                print("Flaw found at line "+row[2])
                ghCodeSnippetHistory_aftercppcheck[idx] = {
                    'repo': ghCodeSnippetHistory['repo'][_],
                    'origin_commit': ghCodeSnippetHistory['origin_commit'][_],
                    'file': ghCodeSnippetHistory['file'][_],
                    'code_context': ghCodeSnippetHistory['code_context'][_],
                    'commit_date': ghCodeSnippetHistory['commit_date'][_],
                    'line_vuln': row[2],
                    'code_vuln': row[4]
                }
                idx = idx + 1
            else:
                print("Syntax error")
                break;
    else:
        ghCodeSnippetHistory_aftercppcheck[idx] = {
            'repo': ghCodeSnippetHistory['repo'][_],
            'origin_commit': ghCodeSnippetHistory['origin_commit'][_] ,
            'file': ghCodeSnippetHistory['file'][_] ,
            'code_context': ghCodeSnippetHistory['code_context'][_], 
            'commit_date': ghCodeSnippetHistory['commit_date'][_],
            'line_vuln': '',
            'code_vuln': ''
        }
        idx = idx + 1
    num = num + 1
    print() 

0
d90054e7c0f41f4bab81df0548577a73b939a87a
tensorflow/core/framework/variant_op_registry.h
b'Checking tensorflow\\core\\framework\\variant_op_registry.h ...\r\n'
b'398;tensorflow\\core\\framework\\variant_op_registry.h;345;style;  UnaryVariantDecodeRegistration(const string& type_name) {\r\n  ^\r\n'
Flaw found at line 345

1
7144571f2fc59c8705e4e3d7b922fa0ebf44f3fa
tensorflow/core/framework/variant_op_registry.h
b'Checking tensorflow\\core\\framework\\variant_op_registry.h ...\r\n'
b'398;tensorflow\\core\\framework\\variant_op_registry.h;345;style;  UnaryVariantDecodeRegistration(const string& type_name) {\r\n  ^\r\n'
Flaw found at line 345

2
7817237d5852c7778d3ba03c40f139e6f2c37a76
tensorflow/core/framework/variant_op_registry.h
b'Checking tensorflow\\core\\framework\\variant_op_registry.h ...\r\n'
b'398;tensorflow\\core\\framework\\variant_op_registry.h;345;style;  UnaryVariantDecodeRegistration(const string& type_name) {\r\n  ^\r\n'
Flaw found at line 345

3
36e1a5ea5ba2dd5eaa7f4cf

In [16]:
ghCodeSnippetHistory_aftercppcheck

{0: {'repo': 'tensorflow/tensorflow',
  'origin_commit': 'd90054e7c0f41f4bab81df0548577a73b939a87a',
  'file': 'tensorflow/core/framework/variant_op_registry.h',
  'code_context': '  // this breaks by falling victim to "too perfect forwarding"  // see https://stackoverflow.com/questions/44475317/variadic-template-issue  // and references therein  template <typename Op>  struct FuncTuple     FuncTuple(const Op& op, const StringPiece& dev, const StringPiece& tname)        : op_type_(op), device_(dev), typename_(tname);    Op op_type_;    StringPiece device_, typename_;  ;  // needed for clang  template <typename Op>    template <typename Op>    std::size_t operator()(const FuncTuple<Op>& x) const       // The hash of an enum is just its value as a std::size_t.      std::size_t ret = static_cast<std::size_t>(x.op_type_);      ret = Hash64Combine(ret, sp_hasher_(x.device_));      ret = Hash64Combine(ret, sp_hasher_(x.typename_));      return ret;      std::unordered_map<FuncTuple<VariantUn

In [17]:
ghCodeSnippetHistory_aftercppcheck = pd.DataFrame.from_dict(ghCodeSnippetHistory_aftercppcheck, orient='index')

In [18]:
ghCodeSnippetHistory_aftercppcheck = ghCodeSnippetHistory_aftercppcheck.reset_index(drop=True)

In [19]:
len(ghCodeSnippetHistory_aftercppcheck)

14

In [20]:
ghCodeSnippetHistory_aftercppcheck

,repo,origin_commit,file,code_context,commit_date,line_vuln,code_vuln
0,tensorflow/tensorflow,d90054e7c0f41f4bab81df0548577a73b939a87a,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-02-07 14:39:49 -0800,345,UnaryVariantDecodeRegistration(const string&...
1,tensorflow/tensorflow,7144571f2fc59c8705e4e3d7b922fa0ebf44f3fa,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-03-12 19:37:39 -0700,345,UnaryVariantDecodeRegistration(const string&...
2,tensorflow/tensorflow,7817237d5852c7778d3ba03c40f139e6f2c37a76,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-08-21 16:22:05 -0700,345,UnaryVariantDecodeRegistration(const string&...
3,tensorflow/tensorflow,36e1a5ea5ba2dd5eaa7f4cfc84a61f8ce3ea20e1,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-09-11 10:51:01 -0700,355,UnaryVariantDecodeRegistration(const string&...
4,tensorflow/tensorflow,6fa6bd045c98bdc89424a3425e15b5161586a9a7,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-10-15 11:01:32 -0700,358,UnaryVariantDecodeRegistration(const string&...
5,tensorflow/tensorflow,3f23f4ddeabbdc0704444d84c158bd6c348a9f10,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2018-10-15 11:22:32 -0700,355,UnaryVariantDecodeRegistration(const string&...
6,tensorflow/tensorflow,52c8bdba0081e4ed428add13e3c0da7ccbfc8f06,tensorflow/core/framework/variant_op_registry.h,,2019-01-26 10:19:26 -0800,314,UnaryVariantDecodeRegistration(const string&...
7,tensorflow/tensorflow,063d42680b33321ee8f334b86c7d37cb19ec0281,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2019-06-20 10:29:14 -0700,314,UnaryVariantDecodeRegistration(const string&...
8,tensorflow/tensorflow,de37b1eaca05431822223e5c996bc08245cf523b,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2020-01-16 13:11:38 -0800,357,UnaryVariantDecodeRegistration(const string&...
9,tensorflow/tensorflow,775a828aade1b87811875ea0ddfa0554c442654b,tensorflow/core/framework/variant_op_registry.h,"// this breaks by falling victim to ""too per...",2020-02-27 15:42:16 +0900,357,UnaryVariantDecodeRegistration(const string&...


In [21]:
chdir(ROOT_PATH)
file_name = repo_name.replace('/','_')
ghCodeSnippetHistory_aftercppcheck.to_csv('GHCodeSnippetHistory/history_'+file_name+".csv")